# Projet 4 : Anticiper les besoins en consommation électrique de bâtiments
*Pierre-Eloi Ragetly*

Ce projet fait parti du parcours *DataScientist* d'OpenClassrooms.

L'objectif principal est de trouver un modèle permettant de prédire **les émissions de CO2 et la consommation totale d’énergie de bâtiments non destinés à l'habitation.**

Pour cela nous disposons des données de la ville de Seattle pour les années 2015 et 2016. Ces données sont à récupérer sur le site kaggle.

# Partie III : Data modeling

Ce notebook a pour but de présenter le travail effectué sur la modélisation.

Nous commencerons par séparer notre jeu de données en deux parties distinctes:
- Le **training set**, qui va permettre d'entrainer les différents modèles;
- Le **testing set**, qui permettra de déterminer la performance du modèle finale.

Pour ce faire, la méthode `train_test_split()` de la classe *sklearn.model_selection* sera utilisée en réservant 20% des données pour le jeu de test.

Puis les modèles les plus courants seront entraînés et comparés afin de conserver les plus prometteurs. Au préalable, *une recherche par quadrillage* sera effectuée pour automatiser le choix des *hyperparamètres*, et les variables les plus pertinentes seront sélectionnées par **RFE** (Recursive Feature Elimination).

Après sélection des modèles les plus performants, nous affinerons encore les hyperparamètres à l'aide d'une *recherche aléatoire* cette fois ci, et nous en profiterons pour tester la pertinence de la variable *EnergyStarScore*.

Nous analyserons enfin les erreurs des modèles afin de déterminer s'il est pertinent d'utiliser une *méthode d'ensemble*, ie. combiner plusieurs modèles pour construire un modèle plus performant.

Le modèle final obtenu, nous pourrons évaluer sa performance à l'aide du jeu de test.

In [1]:
# Import des librairies usuelles
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import seaborn as sns

In [2]:
# Change some default parameters of matplotlib using seaborn
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'axes.titleweight': 'bold'})
sns.set(style='ticks')
current_palette = sns.color_palette('RdBu')
sns.set_palette(current_palette)

In [3]:
# import data
data = (pd.read_csv('data/data_tr.csv')
          .set_index('OSEBuildingID')
          .drop(columns='ENERGYSTARScore'))
data_ess = (pd.read_csv('data/data_tr.csv')
              .set_index('OSEBuildingID')
              .dropna(subset=['ENERGYSTARScore']))

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modéliser-la-consommation-totale-d’énergie" data-toc-modified-id="Modéliser-la-consommation-totale-d’énergie-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modéliser la consommation totale d’énergie</a></span><ul class="toc-item"><li><span><a href="#Créer-un-jeu-de-test" data-toc-modified-id="Créer-un-jeu-de-test-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Créer un jeu de test</a></span></li><li><span><a href="#Comparaison-des-modèles" data-toc-modified-id="Comparaison-des-modèles-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Comparaison des modèles</a></span></li></ul></li></ul></div>

## Modéliser la consommation totale d’énergie

Pour pouvoir modéliser la consommation totale en énergie il faut au préalable déterminer quelle sera notra valeur cible. Sous nous avons quatre variables potentielles :
- SiteEnergyUse(kBtu)
- SiteEnergyUseWN(kBtu)
- SiteEnergyUse(kBtu)_log
- SiteEnergyUseWN(kBtu)_log

Nous avons vu lors de l'ingénierie des variable qu'il était préférable de prendre la version log pour avoir une distribution se rapprochant d'une distribution normale. On peut donc déjà écarter les deux premières.

Se pose ensuite la question de savoir s'il est préférable de garder la version normalisée ou non normalisée. Pour rappel, la version normalisée et la consommation corrigée en prenant comme référence la température des trentes dernières années. Alors que la version normalisée est la consommation moyenne sur les années 2015 et 2016. Dans le contexte de réchauffement climatique, il est fort à parier que la températures des prochaines années sera plus proche de celles de 2015 et 2016 que de la température des trentes dernières années.

Nous prendrons donc la version non normalisée **SiteEnergyUse(kBtu)_log**.

### Créer un jeu de test

In [4]:
from sklearn.model_selection import train_test_split

X = data.iloc[:, :-6].values
y = data.loc[:, 'SiteEnergyUse(kBtu)_log'].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

Le paramètre *random_state* permet de définir *le germe* (seed) du générateur de nombre aléatoires, afin qu'il génère toujours la même suite d'indices pseudo-aléatoires.

### Comparaison des modèles

Pour comparer les résultats obtenus avec les algorithmes de regression les plus courants (voir liste ci-dessous), nous utiliserons la librairie *Scikit-Learn* ainsi que la librairie *XGBoost*.
- Régression Ridge
- Régression Lasso
- Elastic Net
- Regression SVM
- Arbre de décision
- Forêt aléatoire
- Gradient Boosting
- XG Boost
- Perceptron
- Perceptron multi-couches

Nous utiliserons comme mesure de performance la RMSE par validation croisée.

In [5]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

def compare_models_old(X, y):
    """ Fonction to compare the RMSE get with the
    most common Machine Learning Regressors.
    -----------
    Parameters:
    X: Array
        the array object holding data
    y: Array
        the target
    -----------
    Return:
        DataFrame
    """
    # Create models
    ridge = Ridge()
    lasso = Lasso()
    elastic_net = ElasticNet()
    svm_reg = LinearSVR()
    tree_reg = DecisionTreeRegressor()
    forest_reg = RandomForestRegressor()
    gbrt = GradientBoostingRegressor()
    xgbr = XGBRegressor()
    #perceptron = Perceptron()
    models = [ridge,
              lasso,
              elastic_net,
              #svm_reg,
              tree_reg,
              forest_reg,
              gbrt,
              xgbr]
    cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)
    # Train models
    scores = []
    names = []
    std_rmse = []
    for m in models:
        m.fit(X, y)
        m_scores = cross_val_score(m,X, y,
                                   scoring='neg_root_mean_squared_error',
                                   cv=cv)
        m_scores *= -1
        m_names = type(m).__name__
        scores.append(m_scores.mean())
        std_rmse.append(m_scores.std())
        names.append(m_names)
    # Create the DataFrame
    df = pd.DataFrame({'RMSE_mean': scores, 'RMSE_std': std_rmse}, index=names)
    return df

In [6]:
df_comp =  compare_models_old(X_train, y_train)
df_comp

,RMSE_mean,RMSE_std
Ridge,1.167202,0.733387
Lasso,1.302400,0.028977
ElasticNet,1.274081,0.037091
DecisionTreeRegressor,0.442467,0.022268
RandomForestRegressor,0.311599,0.035480
GradientBoostingRegressor,0.222731,0.035183
XGBRegressor,0.234940,0.030488


In [7]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from functions.ml_modeling import linreg_best_params
from functions.ml_modeling import compare_models

# Create models
ridge = linreg_best_params(X_train, y_train, Ridge())
lasso = linreg_best_params(X_train, y_train, Lasso())
elastic_net = linreg_best_params(X_train, y_train,
                                 ElasticNet(), elastic_net=True)
models = [ridge,
          lasso,
          elastic_net]

# Compare models
df = compare_models(X_train, y_train, models)
df

,RMSE_mean,RMSE_std
Ridge,0.893922,-0.085935
Lasso,1.022195,-0.393602
ElasticNet,0.931141,-0.145105


In [8]:
# linear models with rfe
df = compare_models(X_train, y_train, models, rfe=True)
df              

,RMSE_mean,RMSE_std,N_Features
Ridge,0.890859,-0.086410,26
Lasso,1.020869,-0.392210,11
ElasticNet,0.931523,-0.145867,11


In [11]:
from sklearn.metrics import SCORERS
from sklearn.feature_selection import RFECV

cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)
scoring='neg_mean_squared_error'
scorer = SCORERS[scoring]
selector = RFECV(ridge, cv=cv, scoring=scoring)
selector.fit(X_train, y_train)
n_features = selector.n_features_
selector = RFE(ridge, n_features_to_select=n_features)
selector.fit(X_train, y_train)
mask = selector.support_

# Initialization
scores = []
scores_rfe = []

for train, test in cv.split(X_train):
    X_train_2, X_test_2 = X_train[train], X_train[test]
    y_train_2, y_test_2 = y_train[train], y_train[test]
    scores.append(scorer(ridge.fit(X_train_2, y_train_2), X_test_2, y_test_2))
    scores_rfe.append(scorer(ridge.fit(X_train_2[:, mask], y_train_2), X_test_2[:, mask], y_test_2))

In [12]:
scores_a = np.array(scores)
scores_rfe_a = np.array(scores_rfe)
scores_a = np.sqrt(-scores_a)
scores_rfe_a = np.sqrt(-scores_rfe_a)
print("{:.6f}".format(scores_a.mean()))
print("{:.6f}".format(scores_rfe_a.mean()))

0.893922
0.890859


In [ ]:
print(SCORERS.keys())

In [ ]:
cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)
scoring='neg_root_mean_squared_error'
scorer = SCORERS[scoring]
selector = RFECV(ridge, cv=cv, scoring=scoring)
selector.fit(X_train, y_train)
mask = selector.support_

# Initialization
scores = []
scores_rfe = []

for train, test in cv.split(X_train):
    X_train_2, X_test_2 = X_train[train], X_train[test]
    y_train_2, y_test_2 = y_train[train], y_train[test]
    scores.append(scorer(ridge.fit(X_train_2, y_train_2), X_test_2, y_test_2))
    scores_rfe.append(scorer(ridge.fit(X_train_2[:, mask], y_train_2), X_test_2[:, mask], y_test_2))
    
score = np.array(scores).mean()
score_rfe = np.array(scores_rfe).mean()
print("{:.6f}".format(-score))
print("{:.6f}".format(-score_rfe))

In [ ]:
cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)
scoring='neg_root_mean_squared_error'
scorer = SCORERS[scoring]
selector = RFECV(ridge, cv=cv, scoring=scoring)
selector.fit(X_train, y_train)
mask = selector.support_

# Cross validation
scores = cross_val_score(ridge, X_train, y_train, scoring=scoring, cv=cv)
scores_rfe = cross_val_score(ridge, X_train[:, mask], y_train, scoring=scoring, cv=cv)
    
score = -scores.mean()
score_rfe = -scores_rfe.mean()
print("{:.6f}".format(-score))
print("{:.6f}".format(-score_rfe))

In [ ]:
cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)
scoring='neg_root_mean_squared_error'
scorer = SCORERS[scoring]
selector = RFECV(ridge, cv=cv, scoring=scoring)
#selector.fit(X_train, y_train)
X_rfe = selector.fit_transform(X_train, y_train)

# Cross validation
scores = cross_val_score(ridge, X_train, y_train, scoring=scoring, cv=cv)
scores_rfe = cross_val_score(ridge, X_train[:, mask], y_train, scoring=scoring, cv=cv)
    
score = -scores.mean()
score_rfe = -scores_rfe.mean()
print("{:.6f}".format(-score))
print("{:.6f}".format(-score_rfe))